In [0]:
%sql
use catalog setorfarmaceutico;

In [0]:
%sql
DESCRIBE DETAIL setorfarmaceutico.bronze.produtos_fabricados;

In [0]:
df = spark.table("setorfarmaceutico.bronze.produtos_fabricados")
df.limit(5).show()


%md
##### Descreve o tipo de dado de cada coluna

In [0]:
df.printSchema()

##### **Garantia dos tipos de dados corretos**

In [0]:
from pyspark.sql.functions import col, to_date

df = (
    df.withColumn("nome_produto", col("nome_produto").cast("string"))
      .withColumn("lote_fabricacao", col("lote_fabricacao").cast("string"))
      .withColumn("responsavel_lote", col("responsavel_lote").cast("string"))
      .withColumn("fabrica", col("fabrica").cast("string"))
      .withColumn("local_armazenamento", col("local_armazenamento").cast("string"))
      .withColumn("quantidade_fabricada", col("quantidade_fabricada").cast("int"))
      .withColumn("data_fabricacao", to_date(col("data_fabricacao"), "yyyy-MM-dd"))
      .withColumn("data_validade", to_date(col("data_validade"), "yyyy-MM-dd"))
)


In [0]:
top_produtos = (
    df.groupBy("nome_produto")
      .count()
      .orderBy(col("count").desc())
      .limit(10)
)

top_produtos.show(truncate=False)


##### A produção é bem distribuída entre os 5 produtos, sem concentração excessiva.

In [0]:
from pyspark.sql.functions import avg

media_qtd = df.select(avg(col("quantidade_fabricada"))).first()[0]

print(media_qtd)



##### Cada lote fabrica, em média, 550 unidades por produto.

In [0]:
max_qtd = (
    df.select("nome_produto", "quantidade_fabricada")
      .orderBy(col("quantidade_fabricada").desc())
      .limit(10)
)

max_qtd.show(truncate=False)

##### Todos os top 10 têm quantidade_fabricada = 1000
- Existem lotes máximos padronizados (provavelmente capacidade máxima da máquina).

In [0]:
from pyspark.sql.functions import datediff

# 1. Criar coluna com diferença em dias
df = df.withColumn(
    "validade_dias",
    datediff(col("data_validade"), col("data_fabricacao"))
)

# 2. Calcular a média
validade_media = df.select(avg(col("validade_dias"))).first()[0]

print(validade_media)
print("\nValidade média: 379 dias (aprox. 1 ano e 1 mês)")


In [0]:
validade_top = (
    df.select("nome_produto", "validade_dias")
      .orderBy(col("validade_dias").desc())
      .limit(10)
)

validade_top.show(truncate=False)

##### Maiores validades:
- Produtos com até: 730 dias (2 anos)
- Existem produtos de shelf-life longo, ideal para estoque estratégico.

In [0]:
armazenamento_counts = (
    df.groupBy("local_armazenamento")
      .count()
      .orderBy(col("count").desc())
)

armazenamento_counts.show(truncate=False)

##### Distribuição de estoque extremamente equilibrada entre os três armazéns.
- Isso indica boa estratégia de balanceamento logístico.

In [0]:
%sql
use schema silver;

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.stg_produtos_fabricados")


#### Resumo

- Dataset contém 100 mil linhas e 8 atributos relacionados à produção.

- Produção dos produtos é equilibrada, sem dependência de um único item.

- Quantidade média fabricada por lote é 551 unidades.

- Lotes de maior produção atingem 1000 unidades, sugerindo limite de capacidade.

- Validade média de produtos é aprox. 1 ano; alguns chegam a 2 anos.

- Armazenamento é uniformemente distribuído entre Armazém 1, 2 e 3.

- O dataset permite análises de produção, capacidade fabril, shelf-life e logística.